# [모듈 2.1] SageMaker 내장 알고리즘을 위한 이미지 전처리 
Download | Structure | **Preprocessing (Built-in)** | Train Model (Built-in) (4단계 중의 3/4)



### [알림] <font coler="red"> conda_mxnet_latest_p37 커널 </font> 과 함께 사용해야 합니다.
 
* 이 노트북은 `1.1.download_data` 및 `1.2.structuring_data`로 시작하는 일련의 노트북의 일부입니다. 여기에서는 SageMaker의 내장 알고리즘을 위한 데이터 처리에 중점을 둘 것입니다. 이 시리즈의 다음 노트북은 '2.2.builtin_training'입니다.


### 참고
- 세이지 메이커 내장 이미지 분류 알고리즘 --> [SageMaker built-in Image Classification Algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html)

# 노트북 요약
---

- 이 노트북에서는 SageMaker의 기본 제공 알고리즘에 대한 이미지 데이터 세트의 형식을 지정하는 다양한 방법을 살펴봅니다. 
    - (1) Application/x-image format
    - (2) Application/x-recordio (권장 format)
- 우리는 권장 포맷으로서 이미지 데이터 및 레이블이 포함된 던일 바이너리 파일인 .REC 파일(RecordIO 형식)을 생성 합니다.
- RecordIO 형식 (.rec)인 파일을 S3에 업로드합니다.

# 0. 환경 설정

### 카테고리 레이블 로딩

`category_labels` 파일은 이 시리즈 `1.1.download_data.ipynb`의 첫 번째 노트북에서 생성되었습니다. 여기에서 코드를 실행하기 전에 해당 노트북을 실행해야 합니다.

In [1]:
import uuid
import boto3
import shutil
import urllib
import pickle
import pathlib
import sagemaker
import subprocess

In [2]:
with open("pickled_data/category_labels.pickle", "rb") as f:
    category_labels = pickle.load(f)

# 1.  Application/x-image format
___

- 이 형식은 "이미지 형식" 또는 "LST" 형식이라고도 합니다. 이를 사용하면 데이터세트를 수정하거나 재구성할 필요가 없다는 이점이 있습니다. 
- 대신 훈련 세트 및 검증 세트에 대한 이미지의 매니페스트 파일을 생성합니다. (예: train.lst)
- 이 두 매니페스트는 별도의 `.lst` 파일로, 각 이미지에 아래의 세 가지로 구성이 됩니다.
    - (1) 고유 이미지 인덱스
    - (2) 이미지의 카테고리 (레이블)
    - (3) 기본 훈련 폴더의 이미지 파일에 대한 상대 경로
    
    
- `.lst` 파일의 데이터는 탭으로 구분된 값에 있습니다.

- 사용하기 가장 쉬운 형식이지만 SageMaker가 뒤에서 더 많은 작업을 수행해야 합니다. 이미지가 많은 데이터 세트의 경우 학습 시간이 더 오래 걸립니다. 더 적은 수의 이미지가 있는 데이터세트의 경우 성능 차이가 그렇게 뚜렷하지 않습니다.

- 다음은 .LST 매니페스트 파일을 만드는 방법에 대한 두 가지 예입니다. 
    - Option 1: 하나는 자체 코드를 사용하고 
    - Option 2:  MXNet의 정의된 스크립트를 사용합니다. 
        - 실제로 아래에서 .REC 파일을 생성시에는 옵션 2를 사용합니다. 

## Option 1: 직접 코딩으로 생성한 the .LST files

In [3]:
category_ids = {name: idx for idx, name in enumerate(sorted(category_labels.values()))}
print(category_ids)

{'bear': 0, 'bird': 1, 'cat': 2, 'cow': 3, 'dog': 4, 'elephant': 5, 'frog': 6, 'giraffe': 7, 'horse': 8, 'sheep': 9, 'zebra': 10}


In [4]:
import os

train_lst_name = 'train.lst' ; val_lst_name = 'val.lst' ; test_lst_name = 'test.lst'


def delete_file(file_name):
    if os.path.isfile(file_name):
        os.remove(file_name)
        print(f"{file_name} is deleted")
        
# 기존에 lst 파일이 있으면 삭제 합니다.        
delete_file(train_lst_name)        
delete_file(val_lst_name)        
delete_file(test_lst_name)        

train.lst, val.lst, test.lst 파일을 생성 합니다.

In [5]:
image_paths = pathlib.Path("./data_structured").rglob("*.jpg")

for idx, p in enumerate(image_paths): # p: data_structured/test/giraffe/000000354291.jpg
    image_id = f"{idx:010}" # idx 숫자 앞에 0을 채우면서 총 10자리의 문자를 생성 (에: 0000000001)
    category = category_ids[p.parts[-2]] # category 추출하여 ID 추출 (예: giraffe --> 7)
    path = p.as_posix()
    split = p.parts[-3] # train, val, test 중의 하나임. (예: p: data_structured/test/giraffe/000000354291.jpg --> test)
    with open(f"{split}.lst", "a") as f: # 예로 train.lst 파일에 쓰기
        line = f"{image_id}\t{category}\t{path}\n" # image_id, category_id, 이미지 경로 지정
        f.write(line)

생성된 파일의 내용을 확인 합니다.

In [6]:
!head -n3 test.lst

0000000000	7	data_structured/test/giraffe/000000354291.jpg
0000000001	7	data_structured/test/giraffe/000000214701.jpg
0000000002	7	data_structured/test/giraffe/000000564688.jpg


In [7]:
!head -n3 train.lst

0000000275	7	data_structured/train/giraffe/000000507557.jpg
0000000276	7	data_structured/train/giraffe/000000185716.jpg
0000000277	7	data_structured/train/giraffe/000000473952.jpg


In [8]:
!head -n3 val.lst

0000002475	7	data_structured/val/giraffe/000000464178.jpg
0000002476	7	data_structured/val/giraffe/000000444464.jpg
0000002477	7	data_structured/val/giraffe/000000513099.jpg


## Option 2: lst 파일을 생성하기 위해 im2rec.py script 이용하기

In [9]:
script_url = "https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py"
urllib.request.urlretrieve(script_url, "im2rec.py");

아래와 같은 형식으로 생성 합니다.

`python im2rec.py --list --recursive LST_FILE_PREFIX DATA_DIR`
* --list - generate an LST file
* --recursive - looks inside subfolders for image data
* LST_FILE_PREFIX - choose the name you want for the `.lst` file
* DATA_DIR - relative path to directory with the data

In [10]:
!python im2rec.py --list --recursive train data_structured/train

bear 0
bird 1
cat 2
cow 3
dog 4
elephant 5
frog 6
giraffe 7
horse 8
sheep 9
zebra 10


In [11]:
!python im2rec.py --list --recursive val data_structured/val

bear 0
bird 1
cat 2
cow 3
dog 4
elephant 5
frog 6
giraffe 7
horse 8
sheep 9
zebra 10


`train.lst` 파일을 확인 합니다.

In [12]:
!head train.lst

1999	9.000000	sheep/000000574928.jpg
121	0.000000	bear/000000359337.jpg
474	2.000000	cat/000000186635.jpg
877	4.000000	dog/000000187167.jpg
1840	9.000000	sheep/000000090683.jpg
536	2.000000	cat/000000333929.jpg
1769	8.000000	horse/000000422878.jpg
1104	5.000000	elephant/000000262979.jpg
1174	5.000000	elephant/000000460403.jpg
1139	5.000000	elephant/000000362284.jpg


<pre>
</pre>

# 2. Application/x-recordio (권장 format)
___
- 이 형식은 일반적으로 RecordIO라고 합니다. '.rec' 접미사가 있는 각 훈련 및 검증 데이터 세트에 대한 새 파일을 생성합니다. 
- `.rec` 파일은 데이터 세트의 모든 이미지를 포함하는 단일 파일이므로 수천 개의 개별 파일 전송과 관련된 오버헤드 없이 SageMaker 훈련 알고리즘으로 직접 스트리밍할 수 있습니다. 
- 이미지가 많은 데이터 세트의 경우 SageMaker가 훈련 알고리즘을 실행하기 전에 모든 이미지 파일을 다운로드할 필요가 없기 때문에 훈련 시간이 크게 단축됩니다. 
    - S3의 스트리밍으로 데이타를 다운로드 받는 "Pipe" 모드를 사용합니다.
- 'im2rec.py' 스크립트를 사용하면 이미지 크기도 자동으로 조정됩니다. 


## (1) application/x-image 의 Option 2 먼저 실행 하고, 결과인 LST files 을 복사
application/x-image 의 Option 2 가 실행이 안되었다면 먼저 실행 해주세요.

In [13]:
import os
import shutil
new_folder_name = "data_recordio"

# data_recordio 폴더가 존재하면 삭제 합니다.
if os.path.isdir(new_folder_name): 
    shutil.rmtree(new_folder_name)
    print(f"{new_folder_name} is deleted")
    


#### data_recordio 폴더 생성 및 train.lst, val.lst 복사 함.
- RecordIO 파일을 만들시에 lst 파일의 내용을 이용 합니다.


In [14]:
recordio_dir = pathlib.Path("./data_recordio")
recordio_dir.mkdir(exist_ok=True)
shutil.copy("train.lst", "data_recordio/")
shutil.copy("val.lst", "data_recordio/");

## (2) .rec 파일들을 RecordIO 포맷으로 생성하기

아래와 같은 규칙으로 im2rec.py 를 실행 합니다.

`python im2rec.py --resize 224 --quality 90 --num-thread 16 LST_FILE_PREFIX DATA_DIR/`
* **--resize**: 파일을 모두 `.rec` 파일에 저장하기 전에 스크립트가 파일 크기를 조정하도록 합니다. 이미지 분류 알고리즘의 경우 기본 크기는 224x224입니다. 지금 크기를 조정하면 `.rec` 파일의 크기도 줄어듭니다.
* **--quality**: 기본 설정은 이미지 데이터를 압축하지 않고 저장합니다. 약간의 압축을 추가하면 특히 크기를 조정하지 않는 경우 `.rec`의 파일 크기를 작게 유지합니다.
* **--num_thread**: 작업을 병렬화할 스레드 수 설정
* **--LST_FILE_PREFIX**: `.rec` 파일을 생성하기 위해 참조하는 `.lst`의 이름
* **--DATA_DIR**: `.lst` 파일에 나열된 데이터가 있는 상대 경로 디렉토리


훈련 rec 를 생성하기

In [15]:
!python im2rec.py --resize 224 --quality 90 --num-thread 16 data_recordio/train data_structured/train
# !python im2rec.py --resize 224 --quality 90 --num-thread 16 data_recordio/train .

Creating .rec file from /home/ec2-user/SageMaker/ml-data-prep-workshop/image-classificaton/data_recordio/train.lst in /home/ec2-user/SageMaker/ml-data-prep-workshop/image-classificaton/data_recordio
time: 0.028966426849365234  count: 0
time: 0.4975893497467041  count: 1000
time: 0.41789913177490234  count: 2000


검증 rec 를 생성하기

In [16]:
!python im2rec.py --resize 224 --quality 90 --num-thread 16 data_recordio/val data_structured/val
# !python im2rec.py --resize 224 --quality 90 --num-thread 16 data_recordio/val .

Creating .rec file from /home/ec2-user/SageMaker/ml-data-prep-workshop/image-classificaton/data_recordio/val.lst in /home/ec2-user/SageMaker/ml-data-prep-workshop/image-classificaton/data_recordio
time: 0.008013010025024414  count: 0


# 3. S3에 업로드 하기 
___
SageMaker의 내장 알고리즘이 데이터를 학습하려면 S3 버킷에 저장해야 합니다. 

In [17]:
import sagemaker
bucket_name = sagemaker.Session().default_bucket()
print(bucket_name)

sagemaker-ap-northeast-2-057716757052


### .rec 파일을 S3 업로드 하기

In [18]:
built_in_prefix = 'data_prep_workshop/built-in'

In [19]:
s3_uploader = sagemaker.s3.S3Uploader()

data_path = recordio_dir / "train.rec"

train_s3_uri = f's3://{bucket_name}/{built_in_prefix}/data/train'
train_builtin_s3_uri = s3_uploader.upload(
    local_path=data_path.as_posix(), desired_s3_uri= train_s3_uri
)
print("train_builtin_s3_uri: \n", train_builtin_s3_uri)

train_builtin_s3_uri: 
 s3://sagemaker-ap-northeast-2-057716757052/data_prep_workshop/built-in/data/train/train.rec


In [20]:
data_path = recordio_dir / "val.rec"

val_s3_uri = f's3://{bucket_name}/{built_in_prefix}/data/val'
val_builtin_s3_uri = s3_uploader.upload(
    local_path=data_path.as_posix(), desired_s3_uri= val_s3_uri
)
print("val_builtin_s3_uri: \n", val_builtin_s3_uri)

val_builtin_s3_uri: 
 s3://sagemaker-ap-northeast-2-057716757052/data_prep_workshop/built-in/data/val/val.rec


## 데이터 확인

In [21]:
! aws s3 ls {train_builtin_s3_uri} --recursive
! aws s3 ls {val_builtin_s3_uri} --recursive

2022-01-03 09:07:19   60641456 data_prep_workshop/built-in/data/train/train.rec
2022-01-03 09:07:20    7808476 data_prep_workshop/built-in/data/val/val.rec


## 다음 노트북에서 사용할 변수 저장
- 아래는 다음 노트북에서 변수를 사용하기 위해 저장 합니다.

In [22]:
%store bucket_name
%store train_builtin_s3_uri
%store val_builtin_s3_uri

Stored 'bucket_name' (str)
Stored 'train_builtin_s3_uri' (str)
Stored 'val_builtin_s3_uri' (str)



# 4. 다음 단계
훈련 및 검증 데이터가 S3에 업로드되었으므로 다음 노트북은 SageMaker의 내장 이미지 분류 알고리즘을 사용하여 딥 러닝 모델을 훈련하여 동물 이미지를 분류합니다.